In [1]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, experimental, MaxPool2D
from keras.callbacks import History, EarlyStopping 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.random import set_seed
from tensorflow.keras.utils import to_categorical
import plotly.express as px


In [2]:
# images will be classified as cat (0) or dog (1)
categories = ['cats', 'dogs']
directory = '../input/dogs-cats-images/dataset/training_set'

In [3]:
IMG_SIZE = (150, 150)
# list of image and labels
data = []

# get path to each image
for category in categories:
    folder = os.path.join(directory, category)
    label = categories.index(category)
    for img in os.listdir(folder):
        # ignore incompatible images (gifs in this dataset)
        try:
            img_path = os.path.join(folder, img)
            # convert image into array
            img_arr = cv2.imread(img_path)
            # reshape image
            img_arr = cv2.resize(img_arr, IMG_SIZE)
            # putting image and labels into list
            data.append([img_arr, label])
        except Exception as e:
            print(str(e))
            print(img_path)

In [4]:
len(data)

8000

In [5]:
# shuffle the data
random.shuffle(data)
# show first image array
data[0]

[array([[[142, 152, 170],
         [143, 150, 169],
         [145, 150, 168],
         ...,
         [227, 196, 180],
         [223, 195, 179],
         [226, 196, 181]],
 
        [[141, 151, 169],
         [142, 149, 168],
         [145, 151, 170],
         ...,
         [240, 212, 202],
         [232, 204, 194],
         [224, 197, 186]],
 
        [[139, 152, 170],
         [140, 151, 169],
         [144, 151, 170],
         ...,
         [249, 224, 218],
         [218, 187, 183],
         [214, 186, 175]],
 
        ...,
 
        [[115, 113, 125],
         [150, 146, 158],
         [154, 147, 160],
         ...,
         [138, 132, 162],
         [146, 136, 167],
         [136, 141, 162]],
 
        [[148, 144, 156],
         [173, 166, 179],
         [147, 137, 150],
         ...,
         [134, 128, 158],
         [143, 136, 167],
         [137, 143, 166]],
 
        [[164, 157, 170],
         [143, 134, 147],
         [162, 152, 165],
         ...,
         [132, 133, 159],
  

In [6]:
# lists of images and labels
X = []
y = []

for features, labels in data:
    X.append(features)
    y.append(labels)

In [7]:
# convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [8]:
# normalizing data
X = X/255

In [9]:
X.shape

(8000, 150, 150, 3)

In [10]:
# Define the model architecture
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), input_shape=X.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
          optimizer='adam',
          metrics=['accuracy'])

In [11]:
early_stop = EarlyStopping(monitor="val_loss",patience=5,mode="min") # Ensure the model doesn't overfit

history = model.fit(X, y, epochs=15, validation_split=0.1)

Epoch 1/15
225/225 [==============================] - 17s 57ms/step - loss: 1.9552 - accuracy: 0.5643 - val_loss: 0.8441 - val_accuracy: 0.5188
Epoch 2/15
225/225 [==============================] - 12s 53ms/step - loss: 0.5662 - accuracy: 0.7119 - val_loss: 0.6541 - val_accuracy: 0.6137
Epoch 3/15
225/225 [==============================] - 12s 53ms/step - loss: 0.5008 - accuracy: 0.7498 - val_loss: 0.6171 - val_accuracy: 0.6875
Epoch 4/15
225/225 [==============================] - 12s 54ms/step - loss: 0.4473 - accuracy: 0.7992 - val_loss: 0.4975 - val_accuracy: 0.7538
Epoch 5/15
225/225 [==============================] - 12s 53ms/step - loss: 0.3853 - accuracy: 0.8283 - val_loss: 0.4920 - val_accuracy: 0.7975
Epoch 6/15
225/225 [==============================] - 12s 53ms/step - loss: 0.3463 - accuracy: 0.8515 - val_loss: 0.6380 - val_accuracy: 0.7500
Epoch 7/15
225/225 [==============================] - 12s 53ms/step - loss: 0.2849 - accuracy: 0.8812 - val_loss: 0.4960 - val_accuracy:

In [12]:
# Generate generalization metrics
fig = px.line(
history.history, y=['loss', 'val_loss'],
labels={'index': 'epoch', 'value': 'loss'}, 
title='Training History Loss')
fig.show()

In [13]:
fig = px.line(
history.history, y=['accuracy', 'val_accuracy'],
labels={'index': 'epoch', 'value': 'accuracy'}, 
title='Training History Accuracy')
fig.show()